### The Adam algorithm for bivariate function.

In [38]:
import numpy as np
import polars as pl
from sklearn.preprocessing import MinMaxScaler

#Read the dataset, from which we take the features.
dataset = pl.read_csv("WineQuality.csv", separator=';', ignore_errors = True)


#Select the features for the model.
target_var = "quality"
predictor = "pH"


class Adam():
        
    #Note, we assume obj_func takes two, equally-shaped, one-dimensional arrays and two parameters: b0 and b1 (thus n_params = 2)
    def __init__(self,  params:np.ndarray,  X: np.ndarray, y: np.ndarray, alpha:float = 0.001, eps: float = 10**(-8)):
        assert X.shape[0] == y.shape[0], "The first dimensions of the arrays should match!"

        assert type(alpha) is float and alpha > 0, "The alpha parameter needs to be a positive, numerical value."

    
        self.params = params
        self.n_params = params.shape[0]
        
        #Declare the X variable and y variable.
        self.X = X
        self.y = y

        #Define the alpha and eps which stands for: learning-rate and zero-division-preventer respectively.
        self.alpha = alpha
        self.eps = eps

        #Declare the initial state of m and v (first-moment gradient, second-moment gradient).
        self.m = np.zeros(self.n_params)
        self.v = np.zeros(self.n_params)

    def MSE(self, b0: float, b1:float) -> float:
        return np.sum((b1*self.X + b0 - self.y)**2)/self.X.shape[0]


    def FindGradients(self,h:float) -> np.ndarray:
        partial_derivs = np.zeros(self.n_params)
        substractor = self.MSE(self.params[0], self.params[1])

        partial_derivs[0] = (self.MSE(self.params[0]+h, self.params[1]) - substractor)/h
        partial_derivs[1] = (self.MSE(self.params[0], self.params[1]+h) - substractor)/h

        return partial_derivs


    
    def UpdateMoments(self, bet1:float, bet2:float, h:float):
        Gradients = self.FindGradients(h)

        new_m = (bet1*self.m + (1-bet1)*Gradients)/(1-bet1)
        new_v = (bet2*self.v + (1-bet2)*Gradients**2)/(1-bet2)


        return new_m, new_v
    
    def UpdateParameters(self, bet1:float, bet2:float, h:float):
        m_corr, v_corr = self.UpdateMoments(bet1, bet2, h)

        self.params = self.params - self.alpha * m_corr / ( np.sqrt(v_corr) +self.eps)

        return self.params
    
    



def FitTheLineV2(dataset:pl.DataFrame, predictor:str, target_var:str, eps = 0.001) -> tuple[np.ndarray, float]:
    """The function finds the optimal parameters, b0, b1, so that 
    the loss function of linear regressionmodel dataset[target_var] = b0 + b1*dataset[predictor] is minimized"""

    np.random.seed(1)
    B = np.random.uniform(low = -3, high = 3, size = 2)

    #Take the X feature and y feature from the dataset. Then, numpyize it!. Afterward, normalize it!
    MinMaxScaler_X = MinMaxScaler()
    MinMaxScaler_y = MinMaxScaler()

    X = MinMaxScaler_X.fit_transform(dataset.select(predictor).to_numpy())
    y = MinMaxScaler_y.fit_transform(dataset.select(target_var).to_numpy())


    #Declare an instance of the Adam optimizer.
    Adamek = Adam(B, X, y)

    #The following variable indicates the difference between the new value of the function and the old one.
    #If it's  absolute is less than eps, break the loop.
    val_change= float('inf')

    for i in range(2500):
        B_new = Adamek.UpdateParameters(0.9, 0.9, 0.001)

        val_change = abs( Adamek.MSE(B_new[0], B_new[1]) - Adamek.MSE(B[0], B[1]))
        B = B_new



    return B, Adamek.MSE(B[0], B[1])

        


FitTheLineV2(dataset, predictor, target_var, eps = 0.0005)



(array([ 0.54210661, -0.03406147]), 0.026010558160080955)

In [40]:
import numpy as np
#Define some three-variable function
obj_func = lambda X: X[0]**2+X[1]**2



class Adamv2():

    def __init__(self, params:np.ndarray, alpha: float, eps: float = 10**(-8)):
        self.params = params
        self.n_params = params.shape[0]

        #Alpha is a learning rate.
        self.alpha = alpha
        #epsilon will be util when updating the parameters. It prevents deviding by zero.
        self.eps = eps


        self.m = np.zeros(self.n_params)
        self.v = np.zeros(self.n_params)

    

    def FindGradient(self, params:np.ndarray, h:float):
        #The substractor is a term which always in appers in the quotient difference of every parameters.
        substractor = obj_func(params)

        #Define the gradient-vector where all the values of partial derivates will be stored.
        Gradient = np.empty(self.n_params)


        for i in range(self.n_params):
            offset = np.zeros(self.n_params)
            offset[i] += h

            Gradient[i] = (obj_func(params + offset) - substractor)/h

        del substractor
        return Gradient



    def UpdateParams(self, B:np.ndarray):
        #Find the gradient of the objective function.
        Gradient = self.FindGradient(self.params, 0.001)


        #Update the first-moment gradient and second-moment gradient respectively.
        self.m = (B[0]*self.m + (1-B[0])*Gradient)/(1-B[0])
        self.v = (B[1]*self.v +(1-B[1])*Gradient**2)/(1-B[1])


        #In the end, update the parameters.
        return self.params - self.alpha*self.m/(np.sqrt(self.v) + self.eps)

    

    def MainOptimalization(self, diff_bias:float):
        value_change = float('inf')
        
        for i in range(250):
            new_params = self.UpdateParams(np.array([0.4,0.4]))

            self.params = new_params
          
        return self.params
    

                  



Adamek2 = Adamv2(np.array([0.5, 0.5]), 0.01)

print(Adamek2.MainOptimalization(0.01))









[-0.00194853 -0.00194853]
